# Notebook for plotting extracted covariances on BOT data

Initially written 16 Oct 2019 by Craig Lage.
Updates 11-Nov-23


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
from lsst.daf.butler import Butler
from pathlib import Path

# First, get the data from the PTC run

In [ ]:
repo = '/repo/ir2'
butler = Butler(repo, collections=["LSSTCam/raw/all","LSSTCam/calib"])
plot_dir = Path("./plots")
plot_dir.mkdir(exist_ok=True, parents=True)
butler = Butler("/repo/main", collections=["LSSTCam/raw/all","LSSTCam/calib", "u/cslage/bps_13144S"])
exposure=3021120600576
DETECTOR=55 # E2V
RUN = '13144S'
ptcDataset = butler.get('ptc', detector=DETECTOR, exposure=exposure, instrument='LSSTCam')
gains = ptcDataset.gain
means = ptcDataset.finalMeans
xCorrs = ptcDataset.covariances

# Calculate the best fit covariance at each pixel, by fitting a quadratic to the covariance vs flux data.
## The data with PlotDelta=5 is to make a more readable plot.  The data with PlotDelta=8 is used to make the covariance vs pixel distance plot.
## For pixel (0,0) we calculate the deviation from linearity of the variance.


In [ ]:
Cov = np.zeros([len(means.keys()),8, 8])
for PlotDelta in [5, 8]: # Number of pixels to look at
    pdf = PdfPages(str(plot_dir / f"Covariance_vs_Flux_{PlotDelta}_{RUN}_{DETECTOR}.pdf"))
    for ampNum, amp in enumerate(means.keys()):
        gain = gains[amp]
        NumFluxes = int(len(means[amp]))
        fig = plt.figure(figsize = (16,8))
        plt.suptitle(f"Covariance vs Flux: Run {RUN}, Amp {amp}, Det {DETECTOR}", fontsize = 24)
        plt.subplots_adjust(wspace=0.3, hspace=0.6)
        plotcounter = 0
        for jj in range(PlotDelta-1, -1, -1):
            for ii in range(PlotDelta):
                plotcounter += 1
                plt.subplot(PlotDelta, PlotDelta, plotcounter)
                cov = []
                flux = []

                for n in range(NumFluxes):
                    xcorr = xCorrs[amp][n][ii][jj]
                    mean = means[amp][n]
                    if np.isnan(xcorr) or np.isnan(mean):
                        continue
                    if ii == 0 and jj == 0:
                        # Here we plot the deviation from linearity of the
                        # pixel (0,0) varince
                        xcorr = xcorr - mean / gain
                        cov.append(-xcorr)
                    else:
                        cov.append(xcorr)
                    flux.append(mean)
                cov = np.array(cov)
                flux = np.array(flux)

                plt.scatter(flux, cov, color='blue', marker='x', s=50)
                coefs = np.polyfit(flux*flux, cov, 1)
                if PlotDelta == 8:
                    Cov[ampNum, ii, jj] = coefs[0]
                xplot = np.linspace(0,100000, 100)
                yplot = max(0, coefs[0])*xplot*xplot
                plt.plot(xplot,yplot, color = 'red', lw = 2)
                plt.title("Pixel: (%d, %d)"%(ii, jj), fontsize = 12)
                if jj == 0:
                    plt.xlabel("Central Pixel Charge(e-)", fontsize = 12)
                if ii == 0:
                    plt.ylabel("Correlation", fontsize = 12)
                plt.xlim(0,100000)
                plt.xticks([0,50000,100000],fontsize = 12)

                if ii == 0 and jj == 0:
                    plt.yticks([0,10000],fontsize = 12)
                    plt.ylim(-200,30000)
                elif ii == 0 and jj == 1:
                    plt.yticks([0,2000,4000],fontsize = 12)
                    plt.ylim(-100,4000)
                elif ii == 2 and jj == 0:
                    plt.yticks([0,500,1000],fontsize = 12)
                    plt.ylim(-50,1000)
                elif ii == 1 and jj < 2:
                    plt.yticks([0,1000,2000],fontsize = 12)
                    plt.ylim(-100,2000)
                else:
                    plt.yticks([-200,0,200],fontsize = 12)
                    plt.ylim(-50,400)


        pdf.savefig(fig)  # saves the current figure into a pdf page
        plt.close()
    pdf.close()


# Read the simulation results from the files

In [ ]:
def ReadConfigFile(filename):
    # This reads the Poisson simulator config file for
    # the settings that were run
    # and returns a dictionary with the values

    with open(filename,'r') as file:
        lines=file.readlines()
    lines = [ l.strip() for l in lines ]
    lines = [ l.split() for l in lines if len(l) > 0 and l[0] != '#' ]
    for line in lines:
        if line[1] != '=':
            print("Check line: ",line)
            raise IOError("Error reading config file %s"%filename)
    config = {}
    for line in lines:
        try:
            # First get the ordered pairs
            config.update({line[0]:[eval(line[2]), eval(line[3])]})
        except:
            try:
                # Then get the numeric values
                config.update({line[0]:eval(line[2])})
            except:
                try:
                    # Last, get the strings
                    config.update({line[0]:str(line[2])})
                except:
                    pass
    return config

def ReadAreaFile(filename, nx, ny, nxcenter, nycenter, Area_0):
    # This reads the correlation data file
    # and returns an array with the expected correlations
    Area_0 = 99.9974
    area = np.zeros([nx, ny])
    file = open(filename, 'r')
    lines = file.readlines()
    file.close()
    lines.remove(lines[0]) # Strip the title line    
    for line in lines:
        items = line.split()
        i = int(items[0])
        j = int(items[1])
        area[i,j] = float(items[2])

    sim = np.zeros([8,8])
    num = np.zeros([8,8], dtype = int)    
    try:
        for i in range(nx):
            for j in range(ny):
                ii = abs(i - nxcenter)
                jj = abs(j - nycenter)
                sim[jj,ii] += (area[i,j] - Area_0) / Area_0
                num[jj,ii] += 1
        for i in range(8):
            for j in range(8):
                if num[i,j] >0:
                    sim[i,j] /= float(num[i,j])
    except:
        pass
    return [area,sim]


In [ ]:
# Read the .cfg file
home_dir = '/home/c/cslage/u/BOT/corr_sims/'
run = 0
configfile = home_dir + 'pixel-e2v.cfg'
ConfigData = ReadConfigFile(configfile)
outputfilebase = ConfigData["outputfilebase"]
Nx = ConfigData["PixelBoundaryNx"]
Ny = ConfigData["PixelBoundaryNy"]
XCenter = ConfigData["FilledPixelCoords_0_0"][0]
YCenter = ConfigData["FilledPixelCoords_0_0"][1]
PixelSizeX = ConfigData["PixelSizeX"]
PixelSizeY = ConfigData["PixelSizeY"]
NxCenter = int((XCenter - ConfigData["PixelBoundaryLowerLeft"][0]) / PixelSizeX)
NyCenter = int((YCenter - ConfigData["PixelBoundaryLowerLeft"][1]) / PixelSizeY)
Area_0 = 99.9974
NumElec = ConfigData["CollectedCharge_0_0"]

filename = home_dir + outputfilebase +'_%d_Area_E2V'%run + '.dat'
[area,sim] = ReadAreaFile(filename, Nx, Ny, NxCenter, NyCenter, Area_0)

sim /= float(NumElec)

# Now make the plot of covariance vs distance from central pixel

In [ ]:
pdf = PdfPages(str(plot_dir / f"Covariance_vs_Distance_{RUN}_{DETECTOR}.pdf"))
for ampNum, amp in enumerate(means.keys()):
    # Measured covariances
    rs = []
    covs = []
    for ii in range(PlotDelta):
        for jj in range(PlotDelta):
            r2 = (ii)**2 + (jj)**2
            value = Cov[ampNum,ii,jj]
            if ii == 0 and jj == 0:
                rs.append(0.85)
            else:
                rs.append(r2)
            covs.append(value)
    # Simulations
    simrs = []
    simcs = []
    for i in range(8):
        for j in range(8):
            r2 = i**2 + j**2
            value = sim[i,j]
            if i == 0 and j == 0:
                simrs.append(0.85)
                simcs.append(-value)
            elif value > 0.0:
                simrs.append(r2)
                simcs.append(value)
    fig = plt.figure(figsize = (10,8))
    plt.title(f"Covariance Matrix - Run {RUN}, Amp {amp}", fontsize = 24)        
    plt.scatter(rs, covs, label = "Measured")
    plt.scatter(simrs, simcs, marker = 'x', color='red', label = "Simulated")
    plt.yscale('log')
    plt.ylim(1E-10, 1E-5)
    plt.xscale('log')
    plt.xlim(0.8, 100)
    plt.xticks([1.0, 10.0, 100.0], fontsize=18)
    plt.yticks([1E-10, 1E-9, 1E-8, 1E-7, 1E-6, 1E-5], fontsize=18)
    plt.xlabel("$i^2 + j^2$", fontsize = 18)
    plt.ylabel("Covariance or $\delta$ Area/Area", fontsize = 18)
    plt.legend(fontsize=24)
    pdf.savefig(fig)  # saves the current figure into a pdf page
    plt.close()
pdf.close()
